In [91]:
import pandas as pd
import numpy as np
from sklearn import metrics

from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [77]:
data= pd.read_csv('/home/dsc/Documents/Kschool/16_Tutoria_proyectos_1/archive/carInsurance_train.csv')
data['CallDuration']=(pd.to_datetime(arg=data['CallEnd'],format='%H:%M:%S') -
                      pd.to_datetime(arg=data['CallStart'],format='%H:%M:%S')).astype('timedelta64[m]')
data = data.replace(np.nan, 'desconocido', regex=True)
data = data.drop(labels=['CallEnd','CallStart'], axis=1)

data_train, data_test = train_test_split(data, random_state=42)
data_train

,Id,Age,Job,Marital,Education,Default,Balance,HHInsurance,CarLoan,Communication,LastContactDay,LastContactMonth,NoOfContacts,DaysPassed,PrevAttempts,Outcome,CarInsurance,CallDuration
3012,3013,38,unemployed,single,secondary,0,2470,1,0,cellular,4,feb,1,230,1,other,0,2.0
1436,1437,54,retired,divorced,tertiary,0,0,0,0,desconocido,13,may,1,-1,0,desconocido,1,10.0
96,97,48,management,divorced,tertiary,0,657,0,1,cellular,29,jan,2,-1,0,desconocido,0,4.0
43,44,40,admin.,married,secondary,0,2574,1,0,cellular,15,may,1,176,1,failure,1,7.0
1814,1815,43,management,married,tertiary,0,13015,1,0,cellular,21,nov,1,-1,0,desconocido,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,1131,55,desconocido,married,tertiary,0,0,0,0,desconocido,5,jun,1,-1,0,desconocido,0,0.0
1294,1295,44,technician,married,secondary,0,13761,1,0,desconocido,29,may,1,-1,0,desconocido,0,1.0
860,861,41,blue-collar,married,primary,0,246,0,0,desconocido,19,jun,3,-1,0,desconocido,1,11.0
3507,3508,33,technician,married,secondary,0,557,0,0,cellular,4,may,1,-1,0,desconocido,1,6.0


In [108]:
X_train=data_train.drop(labels=['CarInsurance','Id'], axis=1)
y_train=data_train['CarInsurance']

X_test=data_test.drop(labels=['CarInsurance','Id'], axis=1)
y_test=data_test['CarInsurance']

In [109]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

number_columns = X_train.select_dtypes('number').columns

t=[('Job', 
                         OneHotEncoder(handle_unknown='ignore'),
                         ['Job']),
                        ('Marital', 
                         OneHotEncoder(handle_unknown='ignore'),
                         ['Marital']),
                        ('Education', 
                         OneHotEncoder(handle_unknown='ignore'),
                         ['Education']),
                        ('Communication', 
                         OneHotEncoder(handle_unknown='ignore'),
                         ['Communication']),
                        ('Outcome', 
                         OneHotEncoder(handle_unknown='ignore'),
                         ['Outcome']),
                        ('LastContactMonth', 
                         OneHotEncoder(handle_unknown='ignore'),
                         ['LastContactMonth']),
                      ('scaler', StandardScaler(),number_columns)
                          ]

ct = ColumnTransformer(transformers=t, remainder='passthrough')

X_train=ct.fit_transform(X_train)

In [85]:
from sklearn.linear_model import LogisticRegression


In [110]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [111]:
X_test=ct.fit_transform(X_test)
y_pred = lr.predict(X_test)

In [112]:
metrics.confusion_matrix(y_test, y_pred)

array([[528,  70],
       [101, 301]])

In [113]:
metrics.f1_score(y_test,y_pred)

0.778783958602846

In [114]:
from sklearn.tree import DecisionTreeClassifier


In [136]:
dt = DecisionTreeClassifier(max_depth=5)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5)

In [137]:
y_pred = dt.predict(X_test)

In [138]:
metrics.f1_score(y_test,y_pred)

0.7565714285714286

In [140]:
from sklearn.svm import SVC

In [147]:
sv = SVC(probability=True, kernel='poly', degree=2)
sv.fit(X_train, y_train)

SVC(degree=2, kernel='poly', probability=True)

In [148]:
y_pred = sv.predict(X_test)

In [149]:
metrics.f1_score(y_test,y_pred)

0.788627935723115